In [8]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
import numpy as np
import os
import time
import requests
import sys
import warnings

# Suppress FutureWarning about auto_adjust
warnings.filterwarnings('ignore', category=FutureWarning)

print("=" * 80)
print("PHASE 1: Stock Market Analysis Data Collection & Cleaning")
print("=" * 80)

# Create data directories
os.makedirs('data', exist_ok=True)
os.makedirs('data/cleaned', exist_ok=True)

failed_prices = []
failed_fund = []
start_time = datetime.now()

# ---- STEP 1: Get S&P 500 company list ----
print("\n[STEP 1] Downloading S&P 500 company list...")
try:
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()

    tables = pd.read_html(response.text)
    sp500 = None
    for table in tables:
        if 'Symbol' in table.columns:
            sp500 = table
            break

    if sp500 is not None:
        symbols = sp500['Symbol'].tolist()
        sp500.to_csv('data/sp500_companies.csv', index=False)
        with open('data/sp500_symbols.txt', 'w') as f:
            for symbol in symbols:
                f.write(symbol + '\n')
        print(f"✓ Downloaded {len(symbols)} S&P 500 symbols")
    else:
        raise Exception("Could not find Symbol column in Wikipedia tables")
except Exception as e:
    print(f"✗ Error in Step 1: {str(e)}")
    print("  Attempting fallback...")
    try:
        tables = pd.read_html(url)
        sp500 = tables[0]
        symbols = sp500['Symbol'].tolist()
        sp500.to_csv('data/sp500_companies.csv', index=False)
        with open('data/sp500_symbols.txt', 'w') as f:
            for symbol in symbols:
                f.write(symbol + '\n')
        print(f"✓ Fallback successful: {len(symbols)} symbols")
    except Exception as e2:
        print(f"✗ Fallback failed: {str(e2)}")
        sys.exit(1)

# ---- STEP 2: Download historical stock prices ----
print("\n[STEP 2] Downloading historical stock prices (5 years)...")
print("  This will take 15-25 minutes...")

end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)
stock_data = pd.DataFrame()

for i, symbol in enumerate(symbols):
    try:
        # Download with auto_adjust=True for adjusted prices
        df = yf.download(symbol, start=start_date, end=end_date, progress=False, timeout=10, auto_adjust=True)
        
        if df.empty:
            failed_prices.append(symbol)
            continue
        
        # Flatten MultiIndex columns if they exist
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        
        # Ensure 'Adj Close' exists, else create from 'Close'
        if 'Adj Close' not in df.columns and 'Close' in df.columns:
            df['Adj Close'] = df['Close']
        
        df['Symbol'] = symbol
        df['Date'] = df.index
        stock_data = pd.concat([stock_data, df], ignore_index=False)
    except Exception as e:
        failed_prices.append(symbol)
        print(f"  ✗ Failed to download {symbol}: {str(e)[:50]}")

    if (i + 1) % 50 == 0:
        elapsed = (datetime.now() - start_time).total_seconds() / 60
        print(f"  Progress: {i+1}/{len(symbols)} ({(i+1)/len(symbols)*100:.1f}%) - {elapsed:.1f} min elapsed")

if not stock_data.empty:
    stock_data.reset_index(drop=True, inplace=True)
    available_cols = ['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    cols_to_select = [col for col in available_cols if col in stock_data.columns]
    stock_data = stock_data[cols_to_select]
    stock_data.sort_values(['Symbol', 'Date'], inplace=True, ignore_index=True)
    stock_data.to_csv('data/sp500_stock_prices_5years.csv', index=False)
    print(f"\n✓ Stock prices saved ({len(stock_data)} records)")
    print(f"  Downloaded: {len(symbols) - len(failed_prices)} symbols")
    print(f"  Failed: {len(failed_prices)} symbols")
else:
    print("✗ No stock price data collected")
    sys.exit(1)

# ---- STEP 3: Download financial fundamentals (top 100 companies for speed) ----
print("\n[STEP 3] Downloading fundamentals (this is slow - downloading 100 companies)...")
print("  Tip: Increase limit by modifying symbols_to_download list")

symbols_to_download = symbols

fundamentals = []
for i, symbol in enumerate(symbols_to_download):
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        fundamentals.append({
            'Symbol': symbol,
            'Company_Name': info.get('longName', 'N/A'),
            'Sector': info.get('sector', 'N/A'),
            'Industry': info.get('industry', 'N/A'),
            'Market_Cap': info.get('marketCap'),
            'P_E_Ratio': info.get('trailingPE'),
            'Dividend_Yield': info.get('dividendYield'),
            'EPS': info.get('trailingEps'),
            'Revenue': info.get('totalRevenue'),
            'Profit_Margin': info.get('profitMargins'),
            'Debt_to_Equity': info.get('debtToEquity'),
            'Current_Ratio': info.get('currentRatio'),
            'ROE': info.get('returnOnEquity'),
            'ROA': info.get('returnOnAssets'),
            'Beta': info.get('beta'),
            '52_Week_High': info.get('fiftyTwoWeekHigh'),
            '52_Week_Low': info.get('fiftyTwoWeekLow'),
            'Average_Volume': info.get('averageVolume'),
            'Shares_Outstanding': info.get('sharesOutstanding'),
        })
    except Exception as e:
        failed_fund.append((symbol, str(e)[:50]))
    if (i + 1) % 10 == 0:
        print(f"  {i+1}/{len(symbols_to_download)} companies downloaded...", end='\r')
        time.sleep(0.5)

fundamentals_df = pd.DataFrame(fundamentals)

if not fundamentals_df.empty:
    fundamentals_df.to_csv('data/sp500_fundamentals.csv', index=False)
    print(f"\n✓ Fundamentals saved ({len(fundamentals_df)} records)")
    print(f"  Failed: {len(failed_fund)} companies")
else:
    print("\n✗ No fundamentals data collected")

# ---- STEP 4: Download macroeconomic indicators ----
print("\n[STEP 4] Downloading macroeconomic indicators...")

macro_codes = {
    'DGS10': 'US_10Yr_Bond_Yield',
    'CPIAUCSL': 'CPI_All_Urban_Consumers',
    'UNRATE': 'Unemployment_Rate',
    'INDPRO': 'Industrial_Production_Index',
    'T10Y2Y': 'Term_Spread',
    'VIXCLS': 'VIX',
}

macro_data = pd.DataFrame()
for code, name in macro_codes.items():
    try:
        d_fred = pdr.get_data_fred(code, start=start_date, end=end_date)
        if not d_fred.empty:
            d_fred.rename(columns={code: name}, inplace=True)
            d_fred = d_fred.reset_index()
            if 'Date' not in d_fred.columns:
                d_fred.rename(columns={d_fred.columns[0]: 'Date'}, inplace=True)
            if macro_data.empty:
                macro_data = d_fred
            else:
                macro_data = pd.merge(macro_data, d_fred, on='Date', how='outer')
            print(f"  ✓ {name}")
        else:
            print(f"  ✗ No data for {name}")
    except Exception as e:
        print(f"  ✗ Failed {code}: {str(e)[:50]}")

if not macro_data.empty:
    if 'Date' in macro_data.columns:
        macro_data.sort_values('Date', inplace=True)
        macro_data.fillna(method='ffill', inplace=True)
        macro_data.to_csv('data/macroeconomic_indicators.csv', index=False)
        print(f"✓ Macro data saved ({len(macro_data)} records)")
    else:
        print("⚠️ 'Date' column missing in macroeconomic data — skipping save")
else:
    print("⚠️ No macroeconomic data collected")

# ---- STEP 5: Download sector ETF data ----
print("\n[STEP 5] Downloading sector ETF performance...")

sector_etfs = {
    'XLK': 'Technology',
    'XLV': 'Healthcare',
    'XLF': 'Finance',
    'XLE': 'Energy',
    'XLI': 'Industrials',
    'XLY': 'Consumer_Discretionary',
    'XLP': 'Consumer_Staples',
    'XLRE': 'Real_Estate',
    'XLU': 'Utilities',
    'SPY': 'S&P_500_Index',
}

sector_data = pd.DataFrame()
for ticker, name in sector_etfs.items():
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False, timeout=10, auto_adjust=True)
        if not data.empty:
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)
            df = pd.DataFrame({
                'Date': data.index,
                'Ticker': ticker,
                'Sector': name,
                'Open': data['Open'].values,
                'High': data['High'].values,
                'Low': data['Low'].values,
                'Close': data['Close'].values,
                'Volume': data['Volume'].values,
            })
            sector_data = pd.concat([sector_data, df], ignore_index=False)
            print(f"  ✓ {ticker} ({name})")
        else:
            print(f"  ✗ No data for {ticker}")
    except Exception as e:
        print(f"  ✗ Failed {ticker}: {str(e)[:50]}")

if not sector_data.empty:
    sector_data.reset_index(drop=True, inplace=True)
    sector_data.sort_values(['Ticker', 'Date'], inplace=True)
    sector_data.to_csv('data/sector_etf_performance.csv', index=False)
    print("✓ Sector ETF data saved")
else:
    print("⚠️ No sector ETF data collected")

# ---- STEP 6: Data cleaning ----
print("\n[STEP 6] Cleaning and finalizing data for Power BI...")

if os.path.exists('data/sp500_stock_prices_5years.csv'):
    try:
        stock_prices = pd.read_csv('data/sp500_stock_prices_5years.csv')
        stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])
        stock_prices = stock_prices[stock_prices['Close'] > 0]
        stock_prices = stock_prices[stock_prices['High'] >= stock_prices['Low']]
        stock_prices.sort_values(['Symbol', 'Date'], inplace=True)
        stock_prices['Daily_Return'] = stock_prices.groupby('Symbol')['Adj Close'].pct_change()
        stock_prices.to_csv('data/cleaned/stock_prices_cleaned.csv', index=False)
        print("✓ Stock prices cleaned and saved")
    except Exception as e:
        print(f"✗ Error cleaning stock prices: {str(e)}")

if os.path.exists('data/sp500_fundamentals.csv'):
    try:
        fundamentals = pd.read_csv('data/sp500_fundamentals.csv')
        numeric_columns = ['Market_Cap', 'P_E_Ratio', 'Dividend_Yield', 'EPS', 'Revenue',
                          'Profit_Margin', 'Debt_to_Equity', 'Current_Ratio', 'ROE', 'ROA',
                          'Beta', '52_Week_High', '52_Week_Low', 'Average_Volume', 'Shares_Outstanding']
        for col in numeric_columns:
            fundamentals[col] = pd.to_numeric(fundamentals[col], errors='coerce')
        def categorize_market_cap(market_cap):
            if pd.isna(market_cap):
                return 'Unknown'
            elif market_cap >= 10e9:
                return 'Large Cap'
            elif market_cap >= 2e9:
                return 'Mid Cap'
            else:
                return 'Small Cap'
        fundamentals['Market_Cap_Category'] = fundamentals['Market_Cap'].apply(categorize_market_cap)
        fundamentals.to_csv('data/cleaned/fundamentals_cleaned.csv', index=False)
        print("✓ Fundamentals cleaned and saved")
    except Exception as e:
        print(f"✗ Error cleaning fundamentals: {str(e)}")

if os.path.exists('data/macroeconomic_indicators.csv'):
    try:
        macro_indicators = pd.read_csv('data/macroeconomic_indicators.csv')
        macro_indicators['Date'] = pd.to_datetime(macro_indicators['Date'])
        macro_indicators.fillna(method='ffill', inplace=True)
        macro_indicators.to_csv('data/cleaned/macro_indicators_cleaned.csv', index=False)
        print("✓ Macro indicators cleaned and saved")
    except Exception as e:
        print(f"✗ Error cleaning macro indicators: {str(e)}")

if os.path.exists('data/sector_etf_performance.csv'):
    try:
        sector_etfs_data = pd.read_csv('data/sector_etf_performance.csv')
        sector_etfs_data['Date'] = pd.to_datetime(sector_etfs_data['Date'])
        sector_etfs_data.sort_values(['Ticker', 'Date'], inplace=True)
        sector_etfs_data.to_csv('data/cleaned/sector_etfs_cleaned.csv', index=False)
        print("✓ Sector ETFs cleaned and saved")
    except Exception as e:
        print(f"✗ Error cleaning sector ETFs: {str(e)}")

elapsed_total = (datetime.now() - start_time).total_seconds() / 60
print("\n" + "=" * 80)
print("✓ DATA COLLECTION COMPLETE!")
print("=" * 80)
print(f"Total time: {elapsed_total:.1f} minutes")
print("\nCleaned files ready in: data/cleaned/")
print("  1. stock_prices_cleaned.csv")
print("  2. fundamentals_cleaned.csv")
print("  3. macro_indicators_cleaned.csv (optional)")
print("  4. sector_etfs_cleaned.csv (optional)")
print("\nImport the CSV files into Power BI Desktop!")
print("=" * 80)


PHASE 1: Stock Market Analysis Data Collection & Cleaning

[STEP 1] Downloading S&P 500 company list...
✓ Downloaded 503 S&P 500 symbols

[STEP 2] Downloading historical stock prices (5 years)...
  This will take 15-25 minutes...
  Progress: 50/503 (9.9%) - 0.3 min elapsed



1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-11-12 18:14:12.398539 -> 2025-11-11 18:14:12.398539)')


  Progress: 100/503 (19.9%) - 0.6 min elapsed
  Progress: 150/503 (29.8%) - 0.9 min elapsed
  Progress: 200/503 (39.8%) - 1.2 min elapsed
  Progress: 250/503 (49.7%) - 1.5 min elapsed
  Progress: 300/503 (59.6%) - 1.8 min elapsed
  Progress: 350/503 (69.6%) - 2.1 min elapsed
  Progress: 400/503 (79.5%) - 2.5 min elapsed
  Progress: 450/503 (89.5%) - 2.7 min elapsed
  Progress: 500/503 (99.4%) - 3.1 min elapsed

✓ Stock prices saved (621931 records)
  Downloaded: 501 symbols
  Failed: 2 symbols

[STEP 3] Downloading fundamentals (this is slow - downloading 100 companies)...
  Tip: Increase limit by modifying symbols_to_download list
  500/503 companies downloaded...
✓ Fundamentals saved (503 records)
  Failed: 0 companies

[STEP 4] Downloading macroeconomic indicators...
  ✓ US_10Yr_Bond_Yield
  ✓ CPI_All_Urban_Consumers
  ✓ Unemployment_Rate
  ✓ Industrial_Production_Index
  ✓ Term_Spread
  ✓ VIX
✓ Macro data saved (1317 records)

[STEP 5] Downloading sector ETF performance...
  ✓ XLK 